In [1]:
import pandas as pd
import json
import conllu
import numpy as np
import json
import regex as re


In [2]:
class_edges = {
    "root": ["concrete", "abstract", "location", "time", "state", "scalar", "currency", "organization"],
    "concrete": ["int_control", "natural", "phys_obj", "solid", "shape", "substance"],
    "abstract": ["idea", "sound", "communication"],
    "location": ["region", "place", "object"],
    "int_control": ["force", "machine"],
    "natural": ["animate", "plant"],
    "phys_obj": ["comestible", "artifact"],
    "solid": ["rigid", "non-rigid"],
    "shape": ["pointed", "elongated"],
    "machine": ["vehicle"],
    "animate": ["human", "animal", "body_part"],
    "artifact": ["machine", "tool", "garment"]
}


In [3]:
# roles_wide = pd.read_csv("active_passive_decompV2.csv", index_col=0)
# roles_wide.head()
headers = pd.read_csv("decomp_ud_modified_sentences.csv", header=None, nrows=2,
                      index_col=0, keep_default_na=False).values.tolist()
roles_wide = pd.read_csv("decomp_ud_modified_sentences.csv", header=[0, 1], index_col=0)
roles_wide.columns = pd.MultiIndex.from_arrays(headers)

roles_wide.head()


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,tree,sentence,spacy_parse,Arg.Tree,Arg.treepos,Predicate,Arg.Stripped,Arg.Parse,Roleset,Arg
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,(SINV\n (PP-1 (IN From) (NP (DT the) (NNP AP)...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,(S\n (S\n (NP-SBJ+NNP America)\n (VP (V...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP+NN wolf),"(0, 1, 1)",cried,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1


In [4]:
def read_conllu(conll_file, propbank = True):
    data = open(conll_file, encoding="utf-8")
    text = data.read()

    fields = ['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc']
    if propbank:
        fields += ['frame', 'role']

    sentences = conllu.parse(
        text,
        fields = fields
        )
    return sentences

conlls_propbank = {}
conlls = {}

sets = ["train.conllu", "test.conllu", "dev.conllu"]
for s in sets:
    conlls_propbank[s] = read_conllu("UP_English-EWT/en_ewt-up-" + s)


In [5]:
conlls_propbank["train.conllu"][0][0]


{'id': 1,
 'form': 'Al',
 'lemma': 'Al',
 'upos': 'PROPN',
 'xpos': 'NNP',
 'feats': {'Number': 'Sing'},
 'head': 0,
 'deprel': 'root',
 'deps': [('root', 0)],
 'misc': {'SpaceAfter': 'No'},
 'frame': '_',
 'role': '_'}

In [6]:
def get_arg_nums(sentence_id, arg_start_idx, arg_end_idx):
    dataset, index = sentence_id.split(" ")
    dataset = dataset.split("-")[-1]
    index = int(index) - 1

    # iterate through all tokens in arg to find the role
    for i in range(arg_start_idx, arg_end_idx + 1):
        arg = conlls_propbank[dataset][index][i]

        if "role" in arg and "ARG" in arg["role"]:
            return arg["role"][-1]
        
    print("Warning: could not find ARG role ", sentence_id, arg_start_idx, arg_end_idx)
    return np.nan

roles_wide["Arg.ID"] = roles_wide.apply(
                            lambda x: get_arg_nums(
                                x["Sentence.ID"][0],
                                x["Arg.Tokens.Begin"][0],
                                x["Arg.Tokens.End"][0]),
                            axis=1
                        )
print(roles_wide.shape)
# roles_wide = roles_wide.dropna(subset=[("Arg.ID", "")])
roles_wide.head()


(7499, 49)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/2175283176.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["Sentence.ID"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/2175283176.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["Arg.Tokens.Begin"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/2175283176.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x[

,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,sentence,spacy_parse,Arg.Tree,Arg.treepos,Predicate,Arg.Stripped,Arg.Parse,Roleset,Arg,Arg.ID
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,From the AP comes this story :,"{'text': 'From the AP comes this story : ', 's...",(NP-SBJ (DT this) (NN story)),"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP-SBJ+NNP America),"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,"America cried wolf in Iraq , and what 's scary...","{'text': ""America cried wolf in Iraq, and what...",(NP+NN wolf),"(0, 1, 1)",cried,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1,1


In [7]:
# verbnet 3.4 imported from https://uvi.colorado.edu/nlp_applications#download

uvi = json.loads(open("uvi_export.json").read())

pb = {}
for frame in uvi["PropBank"]:
    for predicate in frame["predicates"]:
        for roleset in predicate["rolesets"]:
            roleset_id = roleset["roleset_id"]

            pb[roleset_id] = {}
            args = roleset["roles"]
            for arg in args:
                n = arg["n"]
                if n is None or arg["vnroles"] is None:
                    continue
                
                pb[roleset_id][n] = []
                for vn_mapping in arg["vnroles"]:
                    try:
                        vn_id, vn_role = re.sub(r'([0-9|.|-]+)-([a-z|-]+)',
                                                r'\1__\2', vn_mapping).split("__")
                        pb[roleset_id][n].append((vn_id, vn_role))
                    except:
                        print(vn_mapping)


-result


In [8]:
vn_srestrs = {}
vn = {}
for verb_class in uvi["VerbNet"]:
    class_id = "-".join(verb_class['class_id'].split("-")[1:])
    vn[class_id] = verb_class
    arg_srestrs = {}
    for role in verb_class["themroles"]:
        arg_srestrs[role["themrole"].lower()] = role["selrestrs"]
    vn_srestrs[class_id] = arg_srestrs


### Animacy as a proxy for sentience, instigation, awareness, and volition

In [9]:
def map_to_verbnet(roleset, arg_id):
    try:
        vn_mappings = pb[roleset][arg_id]
        restrs = []
        for vn_id, vn_role in vn_mappings:
            while True:
                if vn_id in vn_srestrs and vn_role in vn_srestrs[vn_id]:
                    r = vn_srestrs[vn_id][vn_role]
                    if r["selrestrs_list"] is not None:
                        restrs.append(vn_srestrs[vn_id][vn_role])
                # check parent classes
                if "-" in vn_id:
                    vn_id = "-".join(vn_id.split("-")[:-1])
                else:
                    break

        return vn_mappings, restrs
    except:
        return np.nan, []
    
roles_wide[["VN_mappings", "Restrictions"]] = roles_wide.apply(lambda x: map_to_verbnet(
                                x["Roleset"][0],
                                x["Arg.ID"][0]),
                            axis=1, result_type="expand"
                        )

print(roles_wide.shape)

roles_wide["VN_mappings"] = roles_wide["VN_mappings"].replace([], np.nan)
roles_wide["has_mapping"] = ~roles_wide["VN_mappings"].isna()
roles_wide.to_csv("decompv2_modified_sentences_verbnet.csv")

roles_wide.head()


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/1969445214.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["Roleset"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/1969445214.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["Arg.ID"][0]),


(7499, 51)


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,Arg.treepos,Predicate,Arg.Stripped,Arg.Parse,Roleset,Arg,Arg.ID,VN_mappings,Restrictions,has_mapping
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,"(2,)",comes,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,"(0, 0)",cried,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,"(0, 1, 1)",cried,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1,1,"[(37.3, topic)]","[{'selrestrs_list': [{'value': '+', 'type': 'c...",True


In [10]:
def is_animate(restrs, strong=False):
    for rs in restrs:
        for r in rs["selrestrs_list"]:
            if r["type"] in ["animate", "human"] \
                and r["value"] == "+":
                if strong:
                    if rs["selrestr_logic"] == "&":
                        return True
                else:
                    return True
    return False

roles_wide["animate_STR"] = roles_wide[("Restrictions", "")].apply(is_animate, strong=True)
roles_wide["animate_WK"] = roles_wide[("Restrictions", "")].apply(is_animate, strong=False)


In [11]:
print(roles_wide.loc[roles_wide["animate_STR"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].mean())
print(roles_wide.loc[roles_wide["animate_STR"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].melt().value_counts().reset_index().sort_values("variable_1"))


Response  sentient       4.600733
          instigation    4.191697
          volition       4.269231
          awareness      4.621490
dtype: float64
   variable_0   variable_1  value  count
1    Response    awareness    5.0    686
18   Response    awareness    2.0      9
16   Response    awareness    3.0     15
9    Response    awareness    4.0     61
11   Response    awareness    1.0     48
10   Response  instigation    2.0     49
3    Response  instigation    5.0    467
4    Response  instigation    4.0    185
7    Response  instigation    3.0     71
12   Response  instigation    1.0     47
17   Response     sentient    3.0     10
14   Response     sentient    4.0     38
0    Response     sentient    5.0    703
8    Response     sentient    1.0     65
19   Response     sentient    2.0      3
13   Response     volition    3.0     41
15   Response     volition    2.0     33
6    Response     volition    1.0     76
5    Response     volition    4.0    112
2    Response     volition   

In [12]:
print(roles_wide.loc[roles_wide["animate_WK"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].mean())
print(roles_wide.loc[roles_wide["animate_WK"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].melt().value_counts().reset_index().sort_values("variable_1"))


Response  sentient       4.525158
          instigation    4.328923
          volition       4.348490
          awareness      4.630451
dtype: float64
   variable_0   variable_1  value  count
0    Response    awareness    5.0   2274
19   Response    awareness    2.0     28
17   Response    awareness    3.0     38
22   Response    awareness    4.5      1
10   Response    awareness    1.0    163
8    Response    awareness    4.0    179
20   Response  instigation    4.5      3
12   Response  instigation    2.0    134
9    Response  instigation    3.0    164
11   Response  instigation    1.0    145
4    Response  instigation    4.0    489
3    Response  instigation    5.0   1748
6    Response     sentient    1.0    243
13   Response     sentient    4.0    123
16   Response     sentient    3.0     46
18   Response     sentient    2.0     29
1    Response     sentient    5.0   2242
5    Response     volition    4.0    336
14   Response     volition    3.0    119
15   Response     volition   

In [27]:
print(roles_wide[roles_wide["has_mapping"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].mean())


Response  sentient       3.547131
          instigation    3.886373
          volition       3.341598
          awareness      3.600000
dtype: float64


In [28]:
print(roles_wide[roles_wide["has_mapping"] & ~roles_wide["animate_WK"]][[
    ("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")
    ]].mean())


Response  sentient       2.352754
          instigation    3.345926
          volition       2.111971
          awareness      2.341602
dtype: float64


In [15]:
roles_long = roles_wide[roles_wide["has_mapping"]].reset_index().melt(
    id_vars=[("Sentence.ID", ""), ("Pred.Lemma", ""), ("Arg.Stripped", ""), ("animate_STR", ""), ("animate_WK", "")],
    value_vars=[("Response", "sentient"), ("Response", "instigation"), ("Response", "volition"), ("Response", "awareness")], 
).drop(columns=["variable_0"])
roles_long.columns = ["Sentence.ID", "Pred.Lemma", "Arg.Stripped", "animate_STR", "animate_WK", "property", "response"]
roles_long = roles_long.loc[roles_long["response"] >= 4]
roles_long[["animate_STR", "property"]].value_counts()


animate_STR  property   
False        instigation    2863
             awareness      2401
             sentient       2349
             volition       2126
True         awareness       747
             sentient        741
             volition        668
             instigation     652
Name: count, dtype: int64

In [16]:
roles_long[["animate_WK", "property"]].value_counts()


animate_WK  property   
True        awareness      2454
            sentient       2365
            volition       2247
            instigation    2240
False       instigation    1275
            sentient        725
            awareness       694
            volition        547
Name: count, dtype: int64

### TODO: Cause as a proxy for instigation

In [17]:
frames = vn["48.1.1"]["frames"]
for frame in frames:
    print(frame["semantics"])


[{'predicate': 'appear', 'args': [{'arg_type': 'Event', 'value': 'e1'}, {'arg_type': 'ThemRole', 'value': 'Theme'}], 'bool': None}]
[{'predicate': 'has_location', 'args': [{'arg_type': 'Event', 'value': 'e1'}, {'arg_type': 'ThemRole', 'value': 'Theme'}, {'arg_type': 'ThemRole', 'value': 'Location'}], 'bool': '!'}, {'predicate': 'appear', 'args': [{'arg_type': 'Event', 'value': 'e2'}, {'arg_type': 'ThemRole', 'value': 'Theme'}], 'bool': None}, {'predicate': 'has_location', 'args': [{'arg_type': 'Event', 'value': 'e3'}, {'arg_type': 'ThemRole', 'value': 'Theme'}, {'arg_type': 'ThemRole', 'value': 'Location'}], 'bool': None}]
[{'predicate': 'has_location', 'args': [{'arg_type': 'Event', 'value': 'e1'}, {'arg_type': 'ThemRole', 'value': 'Theme'}, {'arg_type': 'ThemRole', 'value': 'Location'}], 'bool': '!'}, {'predicate': 'appear', 'args': [{'arg_type': 'Event', 'value': 'e2'}, {'arg_type': 'ThemRole', 'value': 'Theme'}], 'bool': None}, {'predicate': 'has_location', 'args': [{'arg_type': 'E

### Change of state, possession, and location

In [18]:
frames = vn["41.1.1-1"]["frames"]
for frame in frames:
    for f in frame["semantics"]:
        print(f)


{'predicate': 'has_state', 'args': [{'arg_type': 'Event', 'value': 'e1'}, {'arg_type': 'ThemRole', 'value': '?Patient'}, {'arg_type': 'VerbSpecific', 'value': 'V_Final_State'}], 'bool': '!'}
{'predicate': 'take_care_of', 'args': [{'arg_type': 'Event', 'value': 'e2'}, {'arg_type': 'ThemRole', 'value': 'Agent'}, {'arg_type': 'ThemRole', 'value': '?Patient'}], 'bool': None}
{'predicate': 'has_state', 'args': [{'arg_type': 'Event', 'value': 'e3'}, {'arg_type': 'ThemRole', 'value': '?Patient'}, {'arg_type': 'VerbSpecific', 'value': 'V_Final_State'}], 'bool': None}
{'predicate': 'cause', 'args': [{'arg_type': 'Event', 'value': 'e2'}, {'arg_type': 'Event', 'value': 'e3'}], 'bool': None}
{'predicate': 'equals', 'args': [{'arg_type': 'ThemRole', 'value': 'Agent'}, {'arg_type': 'ThemRole', 'value': '?Patient'}], 'bool': None}


In [19]:
roles_wide.head()


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,Arg.Stripped,Arg.Parse,Roleset,Arg,Arg.ID,VN_mappings,Restrictions,has_mapping,animate_STR,animate_WK
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True,False,False
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,this story,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True,False,False
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,America,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,wolf,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1,1,"[(37.3, topic)]","[{'selrestrs_list': [{'value': '+', 'type': 'c...",True,False,False


In [20]:
def get_predicates(VN_mappings):
    if VN_mappings != VN_mappings or not VN_mappings: # no VN_mappings
        return False    
    
    predicates = []
    for vn_id, argument in VN_mappings:
        if vn_id not in vn:
            continue
        frames = vn[vn_id]["frames"]
        arg = {'arg_type': 'ThemRole', 'value': argument.title()}
        for frame in frames:
            for f in frame["semantics"]:
                if arg in f['args']:
                    f["arg_position"] = argument.title()
                    predicates.append(f)

    # if predicates:
    predicates = pd.DataFrame(
        predicates,
        columns=["predicate", "args", "bool", "arg_position"]
        ).drop_duplicates(
            subset=["predicate", "bool", "arg_position"]
        )
        # predicates["predicate"] = predicates["bool"].fillna("") + predicates["predicate"]
        # predicates = predicates.drop(columns=["bool"]).set_index("predicate")["args"].to_dict()
    predicates["bool"] = predicates["bool"].apply(lambda x: x != '!')
        # predicates = predicates.set_index(["predicate", "bool", "arg_position"])["args"].to_dict()
    return predicates

    
roles_wide["predicates"] = roles_wide["VN_mappings"].apply(get_predicates)

# roles_wide = roles_wide.dropna(subset=[("VN_mappings", "")])


print(roles_wide.shape)
# roles_wide = roles_wide.dropna(subset=[("VN_mappings", "")])
roles_wide.head()


(7499, 55)


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,Arg.Parse,Roleset,Arg,Arg.ID,VN_mappings,Restrictions,has_mapping,animate_STR,animate_WK,predicates
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True,False,False,predicate ...
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,"{'text': 'this story ', 'sents': [{'start': 0,...",come.03,1,1,"[(48.1.1, theme)]",[],True,False,False,predicate ...
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True,predicate ...
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,"{'text': 'America ', 'sents': [{'start': 0, 'e...",cry.01,0,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True,predicate ...
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,"{'text': 'wolf ', 'sents': [{'start': 0, 'end'...",cry.01,1,1,"[(37.3, topic)]","[{'selrestrs_list': [{'value': '+', 'type': 'c...",True,False,False,predicate ...


In [21]:
roles_wide["predicates"][4]


,predicate,args,bool,arg_position
0,has_information,"[{'arg_type': 'Event', 'value': 'E'}, {'arg_ty...",True,Topic
1,transfer_info,"[{'arg_type': 'Event', 'value': 'e1'}, {'arg_t...",True,Topic


In [22]:
def undergoes_change_of_location(predicates):
    if not isinstance(predicates, pd.DataFrame): # no predicates
        return False
    if len(predicates[predicates["predicate"] == "appear"]) != 0 :
        return True
    if len(predicates[(predicates["predicate"] == "has_location") & (predicates["bool"])]) != 0 and \
        len(predicates[(predicates["predicate"] == "has_location") & (~predicates["bool"])]) != 0:
        return True
    # if len(predicates[(predicates["predicate"] == "motion") & (predicates["bool"])]) != 0:
    #     return True
    # if len(predicates[(predicates["predicate"] == "body_motion") & (predicates["bool"])]) != 0:
    #     return True
    return False
    
def undergoes_change_of_possession(predicates):
    if not isinstance(predicates, pd.DataFrame): # no predicates
        return False
    if len(predicates[(predicates["predicate"] == "has_possession") & (predicates["bool"]) & \
                      (predicates["arg_position"] == "Theme")]) != 0 and \
        len(predicates[(predicates["predicate"] == "has_possession") & (~predicates["bool"]) & \
                       (predicates["arg_position"] == "Theme")]) != 0:
        return True
    if len(predicates[(predicates["predicate"] == "transfer") & \
                        (predicates["arg_position"].isin(["Theme", "Asset"]))]) != 0:
        return True
    if len(predicates[(predicates["predicate"] == "transfer_info") & \
                        (predicates["arg_position"] == "Topic")]) != 0:
        return True
    else:
        return False


def undergoes_change_of_state(predicates):
    if not isinstance(predicates, pd.DataFrame): # no predicates
        return False
    if len(predicates[(predicates["predicate"] == "has_state") & (predicates["bool"])]) != 0 and \
        len(predicates[(predicates["predicate"] == "has_state") & (~predicates["bool"])]) != 0:
        return True
    if len(predicates[(predicates["predicate"] == "be") & (predicates["bool"])]) != 0 and \
        len(predicates[(predicates["predicate"] == "be") & (~predicates["bool"])]) != 0:
        return True
    if len(predicates[(predicates["predicate"] == "has_physical_form") & (predicates["bool"])]) != 0 and \
        len(predicates[(predicates["predicate"] == "has_physical_form") & (~predicates["bool"])]) != 0:
        return True
    # if len(predicates[(predicates["predicate"] == "has_emotional_state")]) != 0:
        # return True
    if len(predicates[(predicates["predicate"] == "degradation_material_integrity") & (predicates["bool"])]) != 0 and \
        len(predicates[(predicates["predicate"] == "degradation_material_integrity") & (~predicates["bool"])]) != 0:
        return True
    if len(predicates[(predicates["predicate"] == "change_value") & (predicates["arg_position"].isin(["Patient", "Theme"]))]) != 0:
        return True
    else:
        return False


roles_wide["verb_possession"] = roles_wide["predicates"].apply(undergoes_change_of_possession)
roles_wide["verb_location"] = roles_wide["predicates"].apply(undergoes_change_of_location)
roles_wide["verb_state"] = roles_wide["predicates"].apply(undergoes_change_of_state)

print(roles_wide["verb_possession"].value_counts())
print(roles_wide["verb_location"].value_counts())
print(roles_wide["verb_state"].value_counts())
roles_wide.head()


verb_possession
False    7186
True      313
Name: count, dtype: int64
verb_location
False    7222
True      277
Name: count, dtype: int64
verb_state
False    7411
True       88
Name: count, dtype: int64


,Sentence.ID,Pred.Lemma,Arg.Phrase,Pred.Token,Arg.Tokens.Begin,Arg.Tokens.End,Gram.Func,Split,Sent.Grammatical,Annotator.ID,...,Arg.ID,VN_mappings,Restrictions,has_mapping,animate_STR,animate_WK,predicates,verb_possession,verb_location,verb_state
,,,,,,,,,,,...,,,,,,,,,,
0,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,7,...,1,"[(48.1.1, theme)]",[],True,False,False,predicate ...,False,True,False
1,en-ud-dev.conllu 1,come,this story,3,4,5,nsubj,dev,5.0,20,...,1,"[(48.1.1, theme)]",[],True,False,False,predicate ...,False,True,False
2,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,2.0,46,...,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True,predicate ...,False,False,False
3,en-ud-dev.conllu 100,cry,America,1,0,0,nsubj,dev,5.0,15,...,0,"[(37.3, agent)]","[{'selrestrs_list': [{'value': '+', 'type': 'a...",True,False,True,predicate ...,False,False,False
4,en-ud-dev.conllu 100,cry,wolf,1,2,2,dobj,dev,5.0,26,...,1,"[(37.3, topic)]","[{'selrestrs_list': [{'value': '+', 'type': 'c...",True,False,False,predicate ...,True,False,False


In [23]:
print(roles_wide[roles_wide["verb_location"]]["Response"]["change_of_location"].mean())
print(roles_wide["Response"]["change_of_location"].mean())
print()
print(roles_wide[roles_wide["verb_location"]]["Response"]["change_of_location"].value_counts())
print(roles_wide[~roles_wide["verb_location"] & roles_wide["has_mapping"]]["Response"]["change_of_location"].value_counts())
print()
print(roles_wide[roles_wide["verb_location"] & \
                 (roles_wide["Response"]["change_of_location"] > 3)]["Pred.Lemma"].value_counts())
print(roles_wide[~roles_wide["verb_location"] & roles_wide["has_mapping"] & \
                 (roles_wide["Response"]["change_of_location"] > 3)]["Pred.Lemma"].value_counts())


3.6462093862815883
2.5149353247099615

change_of_location
5.0    140
1.0     55
3.0     36
4.0     26
2.0     18
2.5      1
4.5      1
Name: count, dtype: int64
change_of_location
3.0    1519
1.0    1380
2.0     763
4.0     546
5.0     387
1.5       3
3.5       2
2.5       2
4.5       1
Name: count, dtype: int64

Pred.Lemma
take        36
go          21
come        19
send         8
travel       8
put          8
move         6
chase        4
return       4
fly          4
drag         4
bring        4
haul         4
guide        3
file         3
show         3
lead         2
abandon      2
approach     2
walk         2
arrive       2
lay          2
remove       2
speed        2
land         2
carry        1
hold         1
escape       1
vacate       1
commit       1
appear       1
forward      1
squeeze      1
issue        1
drop         1
Name: count, dtype: int64
Pred.Lemma
have         194
love          35
find          24
want          23
know          22
            ... 
rent      

In [24]:
print(roles_wide[roles_wide["verb_possession"]]["Response"]["change_of_possession"].mean())
print(roles_wide["Response"]["change_of_possession"].mean())
print()
print(roles_wide[roles_wide["verb_possession"]]["Response"]["change_of_possession"].value_counts())
print(roles_wide[~roles_wide["verb_possession"] & roles_wide["has_mapping"]]["Response"]["change_of_possession"].value_counts())
print()
print(roles_wide[roles_wide["verb_possession"] & \
                 (roles_wide["Response"]["change_of_possession"] > 3)]["Pred.Lemma"].value_counts())
print(roles_wide[~roles_wide["verb_possession"] & roles_wide["has_mapping"] & \
                 (roles_wide["Response"]["change_of_possession"] > 3)]["Pred.Lemma"].value_counts())


2.547923322683706
1.6647553007067608

change_of_possession
1.0    133
5.0     64
2.0     47
4.0     43
3.0     25
3.5      1
Name: count, dtype: int64
change_of_possession
1.0    3001
2.0     848
3.0     467
5.0     125
4.0     123
1.5       2
2.5       1
Name: count, dtype: int64

Pred.Lemma
offer          16
get            10
find            9
give            8
take            8
order           8
buy             8
purchase        4
sell            4
win             3
receive         3
earn            2
leave           2
spend           2
steal           2
call            2
accept          2
pay             2
lose            1
accecpt         1
announce        1
show            1
communicate     1
note            1
issue           1
admit           1
accumulate      1
extend          1
grab            1
suggest         1
report          1
Name: count, dtype: int64
Pred.Lemma
have       59
be         14
love        7
include     6
need        6
           ..
promise     1
say         1

In [25]:
print(roles_wide[roles_wide["verb_state"]]["Response"]["change_of_state"].mean())
print(roles_wide["Response"]["change_of_state"].mean())
print()
print(roles_wide[roles_wide["verb_state"]]["Response"]["change_of_state"].value_counts())
print(roles_wide[~roles_wide["verb_state"] & roles_wide["has_mapping"]]["Response"]["change_of_state"].value_counts())
print()
print(roles_wide[roles_wide["verb_state"] & \
                 (roles_wide["Response"]["change_of_state"] > 3)]["Pred.Lemma"].value_counts())
print(roles_wide[~roles_wide["verb_state"] & roles_wide["has_mapping"]& \
                 (roles_wide["Response"]["change_of_state"] > 3)]["Pred.Lemma"].value_counts())



4.028409090909091
2.634417922389652

change_of_state
5.0    47
3.0    15
4.0    14
1.0     8
2.0     3
4.5     1
Name: count, dtype: int64
change_of_state
1.0    1428
3.0    1144
4.0     864
2.0     855
5.0     487
3.5       6
4.5       4
2.5       3
1.5       1
Name: count, dtype: int64

Pred.Lemma
launch       7
write        5
make         5
change       4
melt         4
break        4
increase     3
crash        2
build        2
slit         2
activate     2
boggle       2
squeeze      2
devastate    2
shut         2
advance      2
satisfy      2
improve      2
tame         2
create       1
shake        1
operate      1
subside      1
calm         1
divide       1
Name: count, dtype: int64
Pred.Lemma
have      233
think      52
love       50
need       33
be         32
         ... 
wear        1
notice      1
walk        1
steal       1
phone       1
Name: count, Length: 256, dtype: int64


In [26]:
roles_wide[roles_wide["Response"].columns] = roles_wide["Response"]

roles_wide.drop(
    columns=["Response", "Applicable"]
).to_csv("pd_verbnet_semantics.csv")


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_58177/2596009905.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  roles_wide.drop(
